# ReTool notebook for usage in the cromwell container with some viewer

[IGV splash page](http://software.broadinstitute.org/software/igv/) <br>
[Broad Inst IGV docker registry](https://hub.docker.com/r/thephilross/igv/) <br>
[wdl doc: scatter/gather smather](https://software.broadinstitute.org/wdl/documentation/article?id=7614) <br>
```bash
# IGV docker container
# https://hub.docker.com/r/thephilross/igv/dockerfile
FROM ubuntu:14.04

MAINTAINER Philipp Ross, philippross369@gmail.com

ENV DEBIAN_FRONTEND noninteractive

RUN apt-get update && apt-get install -y \
	software-properties-common \
	default-jre \
	wget \
	unzip \
	glib-networking-common

RUN mkdir -p /igv && \
	cd /igv && \
	wget http://www.broadinstitute.org/igv/projects/downloads/IGV_2.3.42.zip && \
	unzip IGV_2.3.42.zip && \
	cd IGV_2.3.42/ && \
	sed -i 's/Xmx2000/Xmx8000/g' igv.sh && \
	cd /usr/bin && \
	ln -s /igv/IGV_2.3.42/igv.sh ./igv

CMD ["/usr/bin/igv"]
```


In [2]:
#                         import libraries, view the content of the externally (locally) mounted directory
import os
import sys
import numpy as np
import pandas as pd

cromwell_dir = '/usr/local/etc'
if os.path.isdir(cromwell_dir) == False:
    print('Cromwell not found:')
else:
    print('Womtool and Cromwell jar files')
    print(os.listdir(cromwell_dir))
    
# external_mount = os.getcwd()
external_mount = '/home/tmp_usr/run_dir'
what_all = os.listdir(external_mount)

Womtool and Cromwell jar files
['cromwell-36.jar', 'womtool-36.jar', 'jupyter']


In [5]:
os.mkdir('/home/tmp_usr/run_dir/arr_opts_test')

In [41]:
%%writefile /home/tmp_usr/run_dir/arr_opts_test/Test32opts.json
{
    "some_message": "nothing was input",
    "a_of_a_of_arrays": "false",
    "a_of_arrays": "false"
}

Overwriting /home/tmp_usr/run_dir/arr_opts_test/Test32opts.json


In [42]:
%%writefile /home/tmp_usr/run_dir/arr_opts_test/T32opts.wdl
###############################################
# Usage:
# java -jar full/path/to/cromwell run /home/tmp_usr/run_dir/arr_opts_test/T32opts.wdl
###############################################

task throw_a_fit {
    String personalized_message
    command {
        echo "whatever ${personalized_message}"
    }
    output {
        String message_impossible = read_string(sdtout())
    }
}

workflow test32opts {
    String some_message
    Boolean a_of_arrays
    Boolean a_of_a_of_arrays
    
    if (!a_of_a_of_arrays && !a_of_arrays) {
        call throw_a_fit as fooie {
            input:
                personalized_message = some_message   
        }
    }
    output {
        fooie.message_impossible
    }
}


Overwriting /home/tmp_usr/run_dir/arr_opts_test/T32opts.wdl


In [43]:
run_string = 'java -jar /usr/local/etc/cromwell-36.jar'
run_string += ' run /home/tmp_usr/run_dir/arr_opts_test/T32opts.wdl'
run_string += ' -i /home/tmp_usr/run_dir/arr_opts_test/Test32opts.json'
print(run_string)
os.system(run_string)

java -jar /usr/local/etc/cromwell-36.jar run /home/tmp_usr/run_dir/arr_opts_test/T32opts.wdl -i /home/tmp_usr/run_dir/arr_opts_test/Test32opts.json


256

In [ ]:
# with open('/home/tmp_usr/run_dir/arr_opts_test/T32opts.wdl', 'r') as fh:
#     lines = fh.read().splitlines()
# for line in lines:
#     print(line)

In [32]:
os.listdir('/home/tmp_usr/run_dir/arr_opts_test')

['Test32opts.json', 'T32opts.wdl']

### Write files in this "perishable directory"
Or upload this notebook to the ../tmp_usr/run_dir and output will be saved in the mounted drive.

In [24]:
%%writefile AAA_json.json
{
    "wf_hello.ImpString": [ 
                          [ ["0:0:0", "0:0:1"], 
                            ["0:1:0","0:1:1"] ],  
                          [ ["1:0:0", "1:0:1"], 
                            ["1:1:0","1:1:1"] ], 
                          [ ["2:0:0", "2:0:1"], 
                            ["2:1:0","2:1:1"] ] ]
}

Writing AAA_json.json


In [33]:
%%writefile AAA_T2IdxArray.wdl
###########################
#              AAA_T2IdxArray.wdl
# Usage:
# java -jar full/path/to/cromwell run AAA_T2IdxArray.wdl -i AAA_json.json
# wf_hello.ImpString = [[["1"]]] 
###########################

task hello {
  String addressee
  Int One
  Int Two
  Int Three
  Int sIx
  command {
    echo "Input String =  ${addressee} With Iterator Index: ${sIx}  [${One}][${Two}][${Three}]"
  }
  output {
    String message = read_string(stdout())
  }
}

workflow wf_hello {

  Array[Array[Array[String]]] ImpString
  Int arr1_size = length(ImpString)
  Int arr2_size = length(ImpString[0])
  Int arr3_size = length(ImpString[0][0])

  Array[Int] Index = range(arr1_size * arr2_size * arr3_size)

  scatter (idx in Index) {

    Int reads_idx = idx % arr3_size

    Int lanes_idx = (idx / arr2_size) % arr2_size

    Int samples_idx = (idx / (arr3_size * arr2_size)) % arr1_size

    call hello as howdy {
      input:
        addressee = ImpString[samples_idx][lanes_idx][reads_idx],
        One = samples_idx,
        Two = lanes_idx,
        Three = reads_idx,
        sIx = idx
    }
  }

  output {
     howdy.message
  }
}


Overwriting AAA_T2IdxArray.wdl


In [34]:
run_string = 'java -jar /usr/local/etc/cromwell-36.jar run AAA_T2IdxArray.wdl -i AAA_json.json'
os.system(run_string)

256

In [35]:
"['956b22e4-6af1-4c20-bd9a-fe25c21e7345']"
os.listdir('cromwell-executions/wf_hello')

['956b22e4-6af1-4c20-bd9a-fe25c21e7345']

In [28]:
ex_path = 'cromwell-executions/wf_hello/956b22e4-6af1-4c20-bd9a-fe25c21e7345/call-howdy'
os.listdir(os.path.join(ex_path, 'shard-0/execution'))

['stdout',
 'script.background',
 'stderr',
 'script.submit',
 'script',
 'stdout.background',
 'rc',
 'stderr.background']

In [29]:

shards = [os.path.join(ex_path, d + '/execution') for d in os.listdir(ex_path)]
print('%i shards found\nwith stdout files:\n\n'%(len(shards)))
for shard in shards:
    with open(os.path.join(shard, 'stdout'), 'r') as fh:
        lines = fh.readlines()
    for line in lines:
        print(line.strip())

12 shards found
with stdout files:


Input String =  2:0:1 With Iterator Index: 9  [2][0][1]
Input String =  1:1:1 With Iterator Index: 7  [1][1][1]
Input String =  0:0:0 With Iterator Index: 0  [0][0][0]
Input String =  0:0:1 With Iterator Index: 1  [0][0][1]
Input String =  1:1:0 With Iterator Index: 6  [1][1][0]
Input String =  2:0:0 With Iterator Index: 8  [2][0][0]
Input String =  2:1:1 With Iterator Index: 11  [2][1][1]
Input String =  2:1:0 With Iterator Index: 10  [2][1][0]
Input String =  0:1:1 With Iterator Index: 3  [0][1][1]
Input String =  1:0:0 With Iterator Index: 4  [1][0][0]
Input String =  1:0:1 With Iterator Index: 5  [1][0][1]
Input String =  0:1:0 With Iterator Index: 2  [0][1][0]
